# Install & import packages

In [1]:
# !pip install folium
!python -m spacy download xx_ent_wiki_sm
!pip install geopandas
!pip install bertopic

2023-04-24 03:27:35.764640: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-24 03:27:36.805474: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 72.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('xx_ent_wiki_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pickle
import pandas as pd
import numpy as np
import re
import spacy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import TextBlob
from sklearn.preprocessing import MinMaxScaler
import string

from bertopic import BERTopic
from nltk.corpus import sentiwordnet as swn
from sklearn.preprocessing import MinMaxScaler
import nltk 
import geopandas as gpd 
import geopy 
import matplotlib.pyplot as plt
from geopy.extra.rate_limiter import RateLimiter
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('sentiwordnet')
import time
import json
import csv
import pandas as pd
# time.sleep(300)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


In [3]:
# mount drive into colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
new_df1 = pd.read_csv('/content/drive/MyDrive/DSCI560/TeamProject-DSCI560/Apify Uploads/dataset.csv')
new_df1 = new_df1[['created_at','full_text']]

new_df1['created_at'] = new_df1['created_at'].str.replace('.000Z', '')

new_df2 = pd.read_csv('/content/drive/MyDrive/DSCI560/TeamProject-DSCI560/Apify Uploads/dataset (1).csv')
new_df2 = new_df2[['created_at','full_text']]
new_df2['created_at'] = new_df2['created_at'].str.replace('.000Z', '')

new_df3 = pd.read_csv('/content/drive/MyDrive/DSCI560/TeamProject-DSCI560/Apify Uploads/dataset_twitter-latest-scraper_2023-04-20_17-11-15-498.csv')
new_df3 = new_df3[['created_at','full_text']]
new_df3['created_at'] = new_df1['created_at'].str.replace('.000Z', '')

new_df = pd.concat([new_df1, new_df2, new_df3], axis=0)

In [5]:
import pickle

# Load the saved model from the file
with open('/content/drive/MyDrive/DSCI560/TeamProject-DSCI560/Models/svm_classification.pkl', 'rb') as f:
    model_svm = pickle.load(f)

# Load the saved model from the file
with open('/content/drive/MyDrive/DSCI560/TeamProject-DSCI560/Models/rf_classification.pkl', 'rb') as f:
    model_rf = pickle.load(f)

# Load the saved model from the file
with open('/content/drive/MyDrive/DSCI560/TeamProject-DSCI560/Models/fnn_classification.pkl', 'rb') as f:
    model_fnn = pickle.load(f)    

# Disaster non disaster classification

In [6]:
stopwords_english = stopwords.words('english')
# convert to lower case
new_df['full_text'] = new_df['full_text'].str.lower()

# remove the HTML and URLs from the reviews (ref: https://stackoverflow.com/questions/45999415/removing-html-tags-in-pandas)
new_df['full_text'] = new_df['full_text'].str.replace(r'<[^<>]*>', '', regex=True)

# remove non-alphabetical characters and punctuations from all tweets.
# print(string.punctuation)
new_df['full_text'] = new_df['full_text'].str.replace('[^a-zA-Z]', ' ').replace('[{}]'.format(string.punctuation), '')

# remove extra spaces from all reviews.
new_df['full_text'] = new_df['full_text'].str.strip()

# perform contractions on the reviews by using the package contractions (ref: https://www.dataquest.io/blog/how-to-clean-and-prepare-your-data-for-analysis/).
# new_df['full_text'] = new_df['full_text'].astype(str).apply(lambda x: contractions.fix(x))

#remove the stop words (ref: https://stackoverflow.com/questions/29523254/python-remove-stop-words-from-pandas-dataframe)
new_df['full_text'] = new_df['full_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords_english)]))

new_df['text_split'] = new_df['full_text'].str.split()
new_df = new_df.dropna()

In [7]:
# with open('/content/drive/MyDrive/DSCI560/TeamProject-DSCI560/Duyen/data/ggvec.npy', 'rb') as f:
#     wv = np.load(f, allow_pickle=True)

import gensim.downloader as api
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

wv = KeyedVectors.load('/content/drive/MyDrive/DSCI560/TeamProject-DSCI560/Duyen/data/gg_wv')

In [8]:
# Create an empty list to store the mean vectors
mean_vectors_infer = []

# Iterate over each row of the DataFrame
for index, row in new_df.iterrows():
    # Convert each list of words into a numpy array of word embeddings
    word_embeddings_infer = []
    for word in row['text_split']:
        try:
            word_embeddings_infer.append(wv[word])
        except KeyError:
            pass
    if word_embeddings_infer == []:
      mean_vector_infer = np.array(wv['unk'])
    else:
      # Calculate the mean vector of the word embeddings along the first axis
      mean_vector_infer = np.array(np.mean(np.array(word_embeddings_infer), axis=0))
    # Append the mean vector to the list
    mean_vectors_infer.append(mean_vector_infer)

In [9]:
X_test_infer = mean_vectors_infer

# Use the loaded model to make predictions on the new dataset
predictions_svm = model_svm.predict(X_test_infer)
predictions_rf = model_rf.predict(X_test_infer)
# predictions_fnn = model_fnn.predict(X_test)

In [10]:
X_test_infer_ = np.array(X_test_infer)

In [11]:
predictions_fnn = model_fnn.predict(X_test_infer_)
predictions_fnn = (predictions_fnn > 0.5).astype(int)


8/8 [==============================] - 0s 2ms/step


In [12]:
flattened_arr = predictions_fnn.ravel()

In [13]:

# tweet_dict1 = {'tweet_text': new_df['full_text'], 'label': predictions_svm, 'when': new_df['created_at']}
# tweet_dict2 = {'tweet_text': new_df['full_text'], 'label': predictions_rf, 'when': new_df['created_at']}
# tweet_dict3 = {'tweet_text': new_df['full_text'], 'label': flattened_arr, 'when': new_df['created_at']}

# # Convert the dictionary to a pandas dataframe
# tweet_df1 = pd.DataFrame(tweet_dict1)
# tweet_df2 = pd.DataFrame(tweet_dict2)
# tweet_df3 = pd.DataFrame(tweet_dict3)


# tweet_df = pd.concat([tweet_df1, tweet_df2, tweet_df3], axis=0)

# # Filter the dataframe to only include tweets with a label of 1
# disaster_df = tweet_df[tweet_df['label'] == 1].reset_index(drop=True)

In [14]:
tweet_df = pd.DataFrame({'tweet_text': new_df['full_text'], 'label_svm': predictions_svm, 'label_rf': predictions_rf, 'label_fnn': flattened_arr, 'when': new_df['created_at']})

In [15]:
tweet_df['label'] = tweet_df['label_svm'] | tweet_df['label_rf'] | tweet_df['label_fnn']

In [16]:
# Filter the dataframe to only include tweets with a label of 1
disaster_df = tweet_df[tweet_df['label'] == 1].reset_index(drop=True)
print(len(disaster_df))

39


In [17]:
# tweet_df.iloc[0]

In [18]:
# tweet_df.iloc[250]

In [19]:
disaster_df.drop_duplicates(subset=['tweet_text', 'label']).reset_index(drop=True)

,tweet_text,label_svm,label_rf,label_fnn,when,label
0,earthquake magnitudo samoa islands region minu...,1,0,0,2023-04-20T21:00:03,1
1,,1,0,0,2023-04-20T20:58:57,1
2,earthquake information new event earthquake us...,0,0,1,2023-04-20T20:58:37,1
3,bir ey hakk nda yarg da bulunmadan nce u soruy...,0,0,1,2023-04-20T21:00:38,1
4,marcorubio cbsmiami notice floods useless gove...,1,0,1,2023-04-20T21:00:28,1
5,estimated million people threatened floods hap...,0,1,0,2023-04-20T21:00:12,1
6,rockville new flood mitigation program reimbur...,0,1,0,2023-04-20T21:00:00,1
7,sti spreading like wildfire flood victims evac...,1,1,1,2023-04-20T20:59:19,1
8,mikehudema stop building flood zones,1,0,0,2023-04-20T20:58:24,1
9,mejenwalton dutchbros nicely done,1,0,0,2023-04-20T20:57:21,1


# Severity & disaster type prediction

In [20]:
with open('/content/drive/MyDrive/DSCI560/TeamProject-DSCI560/Models/disaster_type_classification_model.pkl', 'rb') as file:
    tfidf_2 = pickle.load(file)
    lr_classifier_2 = pickle.load(file)

In [21]:
#load Random forest model for classification
with open('/content/drive/MyDrive/DSCI560/TeamProject-DSCI560/Models/disaster_type_classification_model_1.pkl', 'rb') as file:
    rf_disaster_type = pickle.load(file)


In [22]:
X_test_infer_new = X_test_infer_[tweet_df['label']==1, :]

In [23]:
# use rf_disaster_type. to predict the label of new tweet
X_infer_disaster_label=rf_disaster_type.predict(X_test_infer_new)

In [24]:
new_disaster_tfidf = tfidf_2.transform(disaster_df['tweet_text'])

# Use the lr_classifier to predict the label of the new tweet
new_disaster_label = lr_classifier_2.predict(new_disaster_tfidf)
# disaster_df['severity_score'] = disaster_df['tweet_text'].apply(compute_sentiment)

In [25]:
loaded_topic_model = BERTopic.load("/content/drive/MyDrive/DSCI560/TeamProject-DSCI560/Models/bertopic_model")

# Extract top N words for each topic
N = 10
d= loaded_topic_model.get_topics()
topic_words = [list(zip(*v))[0] for k, v in sorted(d.items())]
# Calculate sentiment score for each word
def get_sentiment_score(word):
    synsets = list(swn.senti_synsets(word))
    if not synsets:
        return 0.0
    pos_score = np.mean([s.pos_score() for s in synsets])
    neg_score = np.mean([s.neg_score() for s in synsets])
    return pos_score - neg_score

word_sentiments = {}
for topic in topic_words:
    for word in topic:
        if word not in word_sentiments:
            word_sentiments[word] = get_sentiment_score(word)

# Calculate sentiment score for each topic
topic_sentiments = {}
for i, topic in enumerate(topic_words):
    sentiment_scores = [word_sentiments[word] for word in topic]
    topic_sentiments[i] = np.mean(sentiment_scores)

# Assign severity score to each topic
topic_severity = {}
for topic, sentiment in topic_sentiments.items():
    if sentiment >= 0.5:
        topic_severity[topic] = 5
    elif sentiment >= 0.25:
        topic_severity[topic] = 4
    elif sentiment >= 0:
        topic_severity[topic] = 3
    elif sentiment >= -0.25:
        topic_severity[topic] = 2
    elif sentiment >= -0.5:
        topic_severity[topic] = 1
    else:
        topic_severity[topic] = 0

# Assign severity score to each tweet based on topic similarity
def get_topic_similarity(tweet, topic_words):
    tweet_words = set(tweet.split())
    similarity_scores = [len(tweet_words.intersection(topic))/len(tweet_words.union(topic)) for topic in topic_words]
    return similarity_scores

tweet_severity = []
for tweet in disaster_df['tweet_text']:
    similarity_scores = get_topic_similarity(tweet, topic_words)
    tweet_severity.append(sum([topic_severity[i]*score for i, score in enumerate(similarity_scores)]))
scaler = MinMaxScaler(feature_range=(0, 5))
normalized_scores = scaler.fit_transform(np.array(tweet_severity).reshape(-1, 1))

# Add severity score as a new column to the tweet data
disaster_df['severity_score'] = normalized_scores


In [26]:
# using RF results instead of logistic regression
# disaster_df['disaster_type'] = new_disaster_label
disaster_df['disaster_type']= X_infer_disaster_label

# Location extraction

In [27]:
nlp_wk = spacy.load('xx_ent_wiki_sm')

text_column = "tweet_text"

# Create an empty list to store the locations for each row
locations = []

# Iterate over each row in the dataframe
for index, row in disaster_df.iterrows():
    # Extract the text from the specified column
    text = row[text_column]
    # Use spaCy to identify all the locations in the text
    doc = nlp_wk(text)
    locs = [[index, ent.text] for ent in doc.ents if ent.label_ == 'LOC']
    # Append either the first location or NaN to the main locations list
    if len(locs) > 0:
        locations.append(locs[0][1])
    else:
        locations.append(np.nan)

# Add the list of locations as a new column to the disaster_df
disaster_df["location"] = locations

# Drop rows with NaN values in the location column
disaster_df.dropna(subset=["location"], inplace=True)

In [28]:
locator = geopy.geocoders.Nominatim(user_agent="mygeocoder")
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
disaster_df["address"] = disaster_df["location"].apply(geocode)

# disaster_df['coordinates'] = disaster_df['address'].apply(lambda loc: tuple(loc.point) if loc else None)
disaster_df['coordinates'] = disaster_df['address'].apply(lambda loc: tuple(loc.point) if loc else (None, None, None))
disaster_df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(disaster_df['coordinates'].tolist(), index=disaster_df.index)
disaster_df.latitude.isnull().sum()
disaster_df = disaster_df[pd.notnull(disaster_df["latitude"])]

In [29]:
disaster_df

,tweet_text,label_svm,label_rf,label_fnn,when,label,severity_score,disaster_type,location,address,coordinates,latitude,longitude,altitude
2,earthquake information new event earthquake us...,0,0,1,2023-04-20T20:58:37,1,0.851506,earthquake,mata utu wallis futuna,"(Matāʻutu, Uvea, Wallis-et-Futuna, 98600, Fran...","(-13.2820419, -176.1740224, 0.0)",-13.282042,-176.174022,0.0
12,marquette county declares local state emergenc...,1,1,1,2023-04-20T20:57:04,1,0.000000,flood,marquette county,"(Marquette County, Michigan, United States, (4...","(46.4481521, -87.6305899, 0.0)",46.448152,-87.630590,0.0
13,integral part turkey heritage antakya orthodox...,0,0,1,2023-04-20T20:52:37,1,0.000000,cyclone,turkey,"(Türkiye, (38.9597594, 34.9249653))","(38.9597594, 34.9249653, 0.0)",38.959759,34.924965,0.0
17,integral part turkey heritage antakya orthodox...,0,0,1,2023-04-20T20:52:37,1,0.000000,cyclone,turkey,"(Türkiye, (38.9597594, 34.9249653))","(38.9597594, 34.9249653, 0.0)",38.959759,34.924965,0.0
22,ramprasad c relative us wanted rivian bought t...,0,1,1,2023-04-20T20:55:30,1,0.802849,hurricane,us,"(United States, (39.7837304, -100.445882))","(39.7837304, -100.445882, 0.0)",39.783730,-100.445882,0.0
28,drunkonsaturday alabama slam lol,1,0,1,2023-04-20T20:53:36,1,0.000000,hurricane,alabama,"(Alabama, United States, (33.2588817, -86.8295...","(33.2588817, -86.8295337, 0.0)",33.258882,-86.829534,0.0
31,think know florida flooded last week seeded st...,0,1,1,2023-04-20T20:52:21,1,0.517967,hurricane,florida,"(Florida, United States, (27.7567667, -81.4639...","(27.7567667, -81.4639835, 0.0)",27.756767,-81.463983,0.0


In [30]:
# Compute the range of the scores
score_range = disaster_df['severity_score'].max() - disaster_df['severity_score'].min()

# Divide the scores into 5 equal-width categories
disaster_df['severity_category'] = pd.cut(disaster_df['severity_score'], bins=5, labels=[1,2,3,4,5])


In [31]:
disaster_df

,tweet_text,label_svm,label_rf,label_fnn,when,label,severity_score,disaster_type,location,address,coordinates,latitude,longitude,altitude,severity_category
2,earthquake information new event earthquake us...,0,0,1,2023-04-20T20:58:37,1,0.851506,earthquake,mata utu wallis futuna,"(Matāʻutu, Uvea, Wallis-et-Futuna, 98600, Fran...","(-13.2820419, -176.1740224, 0.0)",-13.282042,-176.174022,0.0,5
12,marquette county declares local state emergenc...,1,1,1,2023-04-20T20:57:04,1,0.000000,flood,marquette county,"(Marquette County, Michigan, United States, (4...","(46.4481521, -87.6305899, 0.0)",46.448152,-87.630590,0.0,1
13,integral part turkey heritage antakya orthodox...,0,0,1,2023-04-20T20:52:37,1,0.000000,cyclone,turkey,"(Türkiye, (38.9597594, 34.9249653))","(38.9597594, 34.9249653, 0.0)",38.959759,34.924965,0.0,1
17,integral part turkey heritage antakya orthodox...,0,0,1,2023-04-20T20:52:37,1,0.000000,cyclone,turkey,"(Türkiye, (38.9597594, 34.9249653))","(38.9597594, 34.9249653, 0.0)",38.959759,34.924965,0.0,1
22,ramprasad c relative us wanted rivian bought t...,0,1,1,2023-04-20T20:55:30,1,0.802849,hurricane,us,"(United States, (39.7837304, -100.445882))","(39.7837304, -100.445882, 0.0)",39.783730,-100.445882,0.0,5
28,drunkonsaturday alabama slam lol,1,0,1,2023-04-20T20:53:36,1,0.000000,hurricane,alabama,"(Alabama, United States, (33.2588817, -86.8295...","(33.2588817, -86.8295337, 0.0)",33.258882,-86.829534,0.0,1
31,think know florida flooded last week seeded st...,0,1,1,2023-04-20T20:52:21,1,0.517967,hurricane,florida,"(Florida, United States, (27.7567667, -81.4639...","(27.7567667, -81.4639835, 0.0)",27.756767,-81.463983,0.0,4


In [32]:
disaster_df['location_name'] = disaster_df['address'].apply(lambda row: row[0].split(',')[0:5])
disaster_df['location_name'] = disaster_df['location_name'].apply(', '.join)

In [33]:
finalized_df = disaster_df[['disaster_type','location_name','longitude','latitude','severity_category','when']]
finalized_df['verified'] = "?"

In [34]:
finalized_df

,disaster_type,location_name,longitude,latitude,severity_category,when,verified
2,earthquake,"Matāʻutu, Uvea, Wallis-et-Futuna, 98600, F...",-176.174022,-13.282042,5,2023-04-20T20:58:37,?
12,flood,"Marquette County, Michigan, United States",-87.630590,46.448152,1,2023-04-20T20:57:04,?
13,cyclone,Türkiye,34.924965,38.959759,1,2023-04-20T20:52:37,?
17,cyclone,Türkiye,34.924965,38.959759,1,2023-04-20T20:52:37,?
22,hurricane,United States,-100.445882,39.783730,5,2023-04-20T20:55:30,?
28,hurricane,"Alabama, United States",-86.829534,33.258882,1,2023-04-20T20:53:36,?
31,hurricane,"Florida, United States",-81.463983,27.756767,4,2023-04-20T20:52:21,?


# Push data into Firestore

In [35]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

In [36]:
cred = credentials.Certificate('/content/drive/MyDrive/DSCI560/TeamProject-DSCI560/serviceAccountKey.json')
firebase_admin.initialize_app(cred)
db = firestore.client()

In [37]:
# save data from firestore database

# collection_ref = db.collection('users')

# documents = collection_ref.stream()

# # create an empty list to store the data
# data_list = []

# for doc in documents:
#     data = doc.to_dict()
#     data_list.append(data)
#     # do something with the data, e.g. save it to a file
# with open('/content/drive/MyDrive/....json', 'w') as f:
#     json.dump(data_list, f)

# collection_ref = db.collection('emergency1')

# documents = collection_ref.stream()

# # create an empty list to store the data
# data_list = []

# for doc in documents:
#     data = doc.to_dict()
#     # Convert GeoPoint objects to a dictionary
#     for key, value in data.items():
#         if isinstance(value, firestore.GeoPoint):
#             data[key] = {"latitude": value.latitude, "longitude": value.longitude}
   
#     data_list.append(data)
#     # do something with the data, e.g. save it to a file
# with open('/content/drive/MyDrive/DSCI560/....json', 'w') as f:
#     json.dump(data_list, f)


In [38]:
# push data to firestore
# collection_ref = db.collection('emergency1')

# data = [{"severity": "4", "userLocation": {"latitude": 34.84275635, "longitude": -116.1806586}, "location": "San Bernardino Counties", "verified": "true", "time": "2023-04-17T03:23", "emergencyType": "Earthquake"}, {"severity": "4", "location": "San Angelo", "userLocation": {"latitude": 31.40582235, "longitude": -100.466703}, "verified": "true", "time": "2023-04-17T03:23", "emergencyType": "Earthquake"}, {"emergencyType": "Earthquake", "time": "2023-04-17T03:23", "userLocation": {"latitude": 34.9592083, "longitude": -116.419389}, "location": "Berkeley", "severity": "4", "verified": "true"}, {"severity": "5", "verified": "true", "userLocation": {"latitude": 34.0348138, "longitude": -118.4336074}, "emergencyType": "Fire", "time": "2023-04-03T14:31", "location": ""}, {"emergencyType": "Earthquake", "time": "2023-04-17T03:23", "userLocation": {"latitude": 39.78559237, "longitude": -98.7861853}, "location": "Smith", "verified": "true", "severity": "4"}, {"verified": "true", "emergencyType": "Earthquake", "location": "Ringgold", "userLocation": {"latitude": 40.7345334, "longitude": -94.24525638}, "time": "2023-04-17T03:23", "severity": "4"}, {"location": "Madison", "emergencyType": "Earthquake", "userLocation": {"latitude": 34.76318174, "longitude": -86.55099769}, "time": "2023-04-17T03:23", "severity": "4", "verified": "false"}, {"time": "2023-04-18T14:33", "userLocation": {"latitude": 34.0348129, "longitude": -118.4336257}, "verified": "true", "severity": "5", "location": "Las vegas", "emergencyType": "Fire"}, {"emergencyType": "Earthquake", "verified": "false", "location": "Guthrie", "time": "2023-04-17T03:23", "severity": "4", "userLocation": {"latitude": 41.68296122, "longitude": -94.50247879}}, {"time": "2023-04-18T14:32", "verified": "true", "emergencyType": "Fire", "severity": "4", "location": "Los Angeles", "userLocation": {"latitude": 34.0348173, "longitude": -118.4336177}}, {"emergencyType": "Earthquake", "severity": "4", "time": "2023-04-17T03:23", "location": "Black Hawk", "userLocation": {"latitude": 42.46920524, "longitude": -92.30990247}, "verified": "false"}, {"userLocation": {"latitude": 33.55421591, "longitude": 33.55421591}, "emergencyType": "storm", "location": "los angeles", "time": "2022", "verified": "false", "severity": 1}, {"location": "Osceola", "userLocation": {"latitude": 28.06273304, "longitude": -81.14967681}, "emergencyType": "Earthquake", "severity": "4", "verified": "false", "time": "2023-04-17T03:23"}, {"emergencyType": "Flood", "verified": "false", "userLocation": {"latitude": 34.44269248, "longitude": -87.8427826}, "time": "2023-04-16T15:20", "severity": "1", "location": "Franklin"}, {"severity": "4", "location": "Monroe", "userLocation": {"latitude": 31.57127953, "longitude": -87.3645426}, "verified": "false", "time": "2023-04-17T03:23", "emergencyType": "Earthquake"}, {"verified": "false", "time": "2023-04-17T03:23", "location": "Seattle", "emergencyType": "Earthquake", "userLocation": {"latitude": 33.61613912, "longitude": -100.2591841}, "severity": "4"}]
# # Add data for each document to the collection
# for item in data:
#     collection_ref.add(item)


In [39]:
batch = db.batch()

# Get all documents in the 'emergency1' collection
docs = db.collection('emergency1').stream()

# Delete each document in a batched write operation
for doc in docs:
    batch.delete(doc.reference)

# Commit the batched write operation
batch.commit()

[, , , , , , , , , , , ]

In [40]:
# Create Firestore collection reference
collection_ref = db.collection('emergency1')

# Iterate over rows of DataFrame and add each row to Firestore collection
for index, row in finalized_df.iterrows():
    doc_ref = collection_ref.add({
        u'emergencyType': row['disaster_type'],
        u'location': row['location_name'],
        u'severity': row['severity_category'],
        u'time': row['when'],
        u'userLocation': firestore.GeoPoint(row['latitude'], row['longitude']),
        u'verified': row['verified']
    })